In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense
import pandas as pd

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles
pose = mp_pose.Pose()

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def extract_keypoints1(results, pos):
    #Arm
    leftWrist = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightWrist = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftElbow = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightElbow = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftShoulder = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightShoulder = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    #Body
    leftShoulderBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightShoulderBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftHipBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightHipBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftKneeBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightKneeBody = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    #LowerBody
    leftHipLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightHipLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftKneeLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightKneeLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)

    leftAnkleLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    rightAnkleLower = np.array([[res.x, res.y, 1] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    
    golfdataset = pd.DataFrame([[leftWrist[0], leftWrist[1], rightWrist[0], rightWrist[1], leftElbow[0], leftElbow[1], rightElbow[0], rightElbow[1], leftShoulder[0], leftShoulder[1], rightShoulder[0], rightShoulder[1], leftShoulderBody[0], leftShoulderBody[1], rightShoulderBody[0], rightShoulderBody[1], leftHipBody[0], leftHipBody[1], rightHipBody[0], rightHipBody[1], leftKneeBody[0], leftKneeBody[1], rightKneeBody[0], rightKneeBody[1], leftHipLower[0], leftHipLower[1], rightHipLower[0], rightHipLower[1], leftKneeLower[0], leftKneeLower[1], rightKneeLower[0], rightKneeLower[1], leftAnkleLower[0], leftAnkleLower[1], rightAnkleLower[0], rightAnkleLower[1], leftWrist[2]]], columns=['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13', 'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16', 'x17', 'y17', 'x18', 'y18', "Label"])
    golfdataset.to_csv('GolfSEDataset.csv', mode='a', index=False, header=False)
    return 0

In [24]:
# Set up Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Set minimum confidence levels
min_detection_confidence = 0.2
min_tracking_confidence = 0.2

# Define the path to the directory containing the images
image_dir = './Dataset/Takeaway/'

# Get a list of all the image filenames in the directory
image_filenames = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)
                   if os.path.isfile(os.path.join(image_dir, filename))]

# Loop over each image filename
for filename in image_filenames:
    # Read in the image
    image = cv2.imread(filename)
    image = cv2.resize(image,(700,700))

    # Convert the image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect the pose
    with mp_pose.Pose(min_detection_confidence=min_detection_confidence,
                      min_tracking_confidence=min_tracking_confidence) as pose:
        results = pose.process(image)

        # Draw the pose landmarks on the image
        annotated_image = image.copy()
        mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the annotated image
        cv2.putText(annotated_image, "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa", (200,200), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow('Pose Detection', annotated_image)
    if cv2.waitKey(0) & 0xFF == ord('e'):
        cv2.destroyAllWindows()
        break
    # Close the window
cv2.destroyAllWindows()

KeyboardInterrupt: 